In [1]:
import os
from subprocess import run
res = run(["git", "rev-parse", "--show-toplevel"], capture_output=True)
root_dir = res.stdout.decode().strip()
os.chdir(root_dir)

In [168]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%reload_ext autoreload
%autoreload 2

from src.vortector.vortector import Vortector
from src.vortector.visualize import show_fit_overview_2D, show_fit_overview_1D
from src.vortector.contour import ContourDetector

%matplotlib widget

from synthvortex import VortexGenerator


def show_img(arr, ax=None):
    if ax is None:
        fig, ax = plt.subplots(dpi=150)
    else:
        fig = ax.get_figure()
        ax.cla()
    ax.pcolormesh(arr.T)
    return fig, ax

fig_vortector = None

def show_fit(fit):
    low = [x for x in fit["bounds_low"].values()]
    high = [x for x in fit["bounds_high"].values()]
    for n,key in enumerate(["c", "a", "r0", "phi0", "sigma_r", "sigma_phi"]):
        print(f"{key:10s}: {low[n]: 10.5f} < {fit['p0'][n]: 10.5f} < {high[n]: 10.5f}    -> {fit[key]: 10.5f}")

In [169]:
if fig_vortector is None:
    fig_vortector = plt.figure(figsize=(12,8))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [206]:
scaling = 2
# vgen = VortexGenerator(3, periodic_y=True, Nr=121*scaling,Nphi=251*scaling, noise=0, seed=139040831483689419186733474417936403656)
vgen = VortexGenerator(3, periodic_y=True, Nr=121*scaling,Nphi=251*scaling, noise=0, seed=77286521634823927887570432972751377545)
vd = Vortector(vgen.Rs, vgen.Phis, vgen.Area, vgen.vortensity, vgen.density, verbose=False, debug=True)
%time vd.detect(include_mask=True, keep_internals=True)
n = None

show_fit_overview_2D(vd, n=n, fig=fig_vortector)
print("Number of vortices:", len(vd.vortices))
vgen.seed
if n is None:
    vorts = vd.vortices
else:
    vorts = [vd.vortices[n]]
for vort in vorts:
    show_fit(vort["fits"]["surface_density"])
    print("-----")

CPU times: user 898 ms, sys: 334 ms, total: 1.23 s
Wall time: 254 ms
Number of vortices: 2
c         :    0.75146 <    0.80156 <    1.25243    ->    1.01275
a         :    0.65871 <    1.31741 <    2.63482    ->    1.30850
r0        :   10.23570 <   13.04851 <   15.86132    ->   12.99907
phi0      :    3.29179 <    3.69857 <    4.10535    ->   -2.57830
sigma_r   :    0.56256 <    5.62563 <   11.25125    ->    3.53732
sigma_phi :    0.08136 <    0.81356 <    1.62712    ->    0.50195
-----
c         :    0.77004 <    0.82137 <    1.28339    ->    1.00235
a         :    0.35680 <    0.71359 <    1.42719    ->    0.73864
r0        :    2.73558 <    4.96845 <    7.20131    ->    4.95853
phi0      :   -0.52568 <    0.21904 <    0.96376    ->    0.21524
sigma_r   :    0.44657 <    4.46573 <    8.93146    ->    2.40930
sigma_phi :    0.14894 <    1.48944 <    2.97888    ->    0.70161
-----


In [176]:
#%load_ext snakeviz
#%snakeviz vd.detect(include_mask=True, keep_internals=True)

In [186]:
r, phi, vort, dens, mask = vd.extract_data(vd.vortices[0], region="contour")
Z = np.ma.masked_array(dens, mask=~mask)
fig, ax = plt.subplots()
ax.pcolormesh(r, phi, Z)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
fig, ax = show_img(cd.data_extended)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
cd = ContourDetector(vgen.vortensity, np.linspace(-1,1,41))
contours = cd.detect_elliptic_contours()
show_img(cd.data_extended, ax = ax)
for cnt in cd.contours_closed:
# for cnt in contours:
# for cnt in cd.contours:
# for cnt in cd.contours
    try:
        bnd = cnt["boundary"]
    except KeyError:
        bnd = cnt["detection"]["boundary"]
    except TypeError:
        bnd = cd.contours[cnt]["boundary"]
    x = bnd[:,0,1]
    y = bnd[:,0,0]
    ax.plot(x,y)